# Create

In [ ]:
run_hive(
    """
    create table neilpquinn.new_editors(
        user_name string,
        wiki string,
        cohort string,
        1st_month_edits bigint,
        2nd_month_edits bigint
    )
    -- This allows us to delete data later on (e.g. in case of duplicate inserts)
    TBLPROPERTIES ("transactional"="true")
    """
)

# Update

In [1]:
# To-do: We take the metric month and look backwards. But this results in censored data
# for months under 30 days (February).

# Start included, end excluded. Have to set the end back a couple of months to ensure we only get uncensored data.
START = "2008-01"
END = "2015-01"

SNAPSHOT = "2017-12"

In [2]:
# Be careful not to insert duplicate data. Primary keys were introduced in Hive 2.1.0,
# so until we're running that there's no built-in way to stop duplicates.
run_hive(
    """
    insert into table neilpquinn.new_editors
    select 
        1st_month.user_name as user_name,
        1st_month.wiki as wiki,
        1st_month.cohort as cohort,
        1st_month.edits as 1st_month_edits,
        coalesce(2nd_month.edits, 0) as 2nd_month_edits
    from (
        select
            event_user_text as user_name,
            wiki_db as wiki,
            substr(event_user_creation_timestamp, 0, 7) as cohort,
            count(*) as edits
        from wmf.mediawiki_history
        where
            snapshot = "{snapshot}" and
            event_entity = "revision" and
            event_type = "create" and
            event_user_is_created_by_system = 0 and
            event_user_creation_timestamp >= "{start}" and
            event_user_creation_timestamp < "{end}" and
            unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") <
                (unix_timestamp(event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
        group by event_user_text, event_user_creation_timestamp, wiki_db
        ) 1st_month
    left join (
        select
            event_user_text as user_name,
            wiki_db as wiki,
            substr(event_user_creation_timestamp, 0, 7) as cohort,
            count(*) as edits
        from wmf.mediawiki_history
        where
            snapshot = "{snapshot}" and
            event_entity = "revision" and
            event_type = "create" and
            event_user_is_created_by_system = 0 and
            event_user_creation_timestamp >= "{start}" and
            event_user_creation_timestamp < "{end}" and
            unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") >=
                (unix_timestamp(event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
            unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") <
                (unix_timestamp(event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
            group by event_user_text, event_user_creation_timestamp, wiki_db
        ) 2nd_month
    on
        (1st_month.user_name = 2nd_month.user_name and
        1st_month.wiki = 2nd_month.wiki and
        1st_month.cohort = 2nd_month.cohort)
    """.format(
        snapshot = SNAPSHOT,
        start = START,
        end = END
    )
)

# Validate

In [3]:
# Check to make sure no cohorts have duplicated data
run_hive(
    """
    select cohort, count(*)
    from neilpquinn.new_editors
    group by cohort
    """
)

,cohort,_c1
0,2015-01,123555
1,2015-02,123038
2,2015-03,130353
3,2015-04,109273
4,2015-05,110642
5,2015-06,104491
6,2015-07,95070
7,2015-08,94634
8,2015-09,101074
9,2015-10,104745
